In [1]:
import model
from Augment import train_generator

Convolution_Output_size:(5,29)
LSTM_Output_size:(Tensor("bidirectional_rnn/fw/fw/transpose_1:0", shape=(?, 145, 256), dtype=float32),Tensor("ReverseV2:0", shape=(?, 145, 256), dtype=float32))
Found 6161 images belonging to 37 classes.


In [2]:
n_epochs = 10

In [3]:
# !rm -rf ../Augmented/*

In [4]:
epoch = 1
for batch in train_generator:
    if epoch == n_epochs:
        break
    epoch += 1